# Options Pricing using Monte-Carlo Simulation

In [1]:
import numpy as np 
import math
import time


In [4]:
from yahoo_fin import options
chain = options.get_options_chain("GME")
chain['puts']


/home/nate/development/env/lib/python3.11/site-packages/yahoo_fin/options.py:42: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(requests.get(site, headers=headers).text)


,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,GME231124P00005000,2023-11-13 2:26PM EST,5.0,0.04,0.00,0.01,0.00,-,10,13,350.00%
1,GME231124P00007500,2023-10-20 11:29AM EST,7.5,0.07,0.00,0.10,0.00,-,4,4,293.75%
2,GME231124P00008500,2023-11-14 10:40AM EST,8.5,0.01,0.00,0.17,0.00,-,-,3,262.50%
3,GME231124P00009000,2023-11-17 3:01PM EST,9.0,0.01,0.00,0.01,0.00,-,81,94,143.75%
4,GME231124P00009500,2023-11-20 12:14PM EST,9.5,0.01,0.00,0.01,-0.01,-50.00%,1,27,125.00%
5,GME231124P00010000,2023-11-20 3:11PM EST,10.0,0.02,0.01,0.03,-0.01,-33.33%,20,104,128.13%
6,GME231124P00010500,2023-11-20 2:58PM EST,10.5,0.01,0.00,0.03,0.00,-,25,13,101.56%
7,GME231124P00011000,2023-11-20 3:56PM EST,11.0,0.03,0.01,0.03,0.00,-,847,593,85.94%
8,GME231124P00011500,2023-11-20 3:57PM EST,11.5,0.02,0.03,0.04,-0.05,-71.43%,547,309,73.44%
9,GME231124P00012000,2023-11-20 3:56PM EST,12.0,0.05,0.05,0.06,-0.03,-37.50%,824,1047,57.81%


In [1]:
class OptionPricing:

    def __init__(self, S0, E, T, rf, sigma, iterations):
        self.S0 = S0
        self.E = E
        self.T = T
        self.rf = rf 
        self.sigma = sigma 
        self.iterations = iterations 

    def call_option_simulation(self):

        #we have 2 columns: first with 0s, second will store the payoff
        #we need the first column of 0s: payoff function is max(S-E, 0) for call option
        option_data = np.zeros([self.iterations, 2])

        #dimensions: 1-dimensional array with as many items as the iterations
        rand = np.random.normal(0, 1, [1, self.iterations])

        #equation for the S(t) stock price
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2) + self.sigma *np.sqrt(self.T) * rand)

        #we need S-E in order to calculate the max(S-E, 0)
        option_data[:,1] = stock_price - self.E 

        #average for the Monte-Carlo method
        #np.amax() returns the max(S-E, 0) according to the formula
        average = np.sum(np.amax(option_data, axis=1))/float(self.iterations)

        #we have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average

    def put_option_simulation(self):

        #we have 2 columns: first with 0s, second will store the payoff
        #we need the first column of 0s: payoff function is max(S-E, 0) for call option
        option_data = np.zeros([self.iterations, 2])

        #dimensions: 1-dimensional array with as many items as the iterations
        rand = np.random.normal(0, 1, [1, self.iterations])

        #equation for the S(t) stock price
        stock_price = self.S0 * np.exp(self.T * (self.rf - 0.5 * self.sigma**2) + self.sigma *np.sqrt(self.T) * rand)

        #we need S-E in order to calculate the max(E-S, 0)
        option_data[:,1] = self.E - stock_price

        #average for the Monte-Carlo method
        #np.amax() returns the max(E-S, 0) according to the formula
        average = np.sum(np.amax(option_data, axis=1))/float(self.iterations)

        #we have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average



In [2]:
if __name__ == "__main__":

    S0 = 100                #underlying stock price at t=0
    E = 100                 #strike price
    T = 1                   #expiry
    rf = 0.05               #risk-free rate
    sigma = 0.2             #volatility of underlying stock
    iterations = 10000000   #number of iterations in the monte-carlo simulation

    model = OptionsPricing(S0, E, T, rf, sigma, iterations)
    print("Monte-Carlo Call option price: ", model.call_option_simulation())
    print("Monte-Carlo Put option price: ", model.put_option_simulation())